In [15]:
import numpy as np
import random
import joblib
import scipy.io as scipy
import tensorflow as tf

# spilting the data

In [16]:
csi_pairs = joblib.load("csi_up_pairs_100.joblib")
labels_pairs = joblib.load("labels_up_pairs_100.joblib")

In [17]:
csi_pairs.shape, labels_pairs.shape

((10000, 2, 200, 30), (10000, 2, 1))

In [18]:
temp = list(zip(csi_pairs, labels_pairs))
random.shuffle(temp)
csi_pairs, labels_pairs = zip(*temp)
csi_pairs, labels_pairs = list(csi_pairs), list(labels_pairs)

In [19]:
csi_pairs = np.array(csi_pairs)
labels_pairs = np.array(labels_pairs)

In [20]:
y_train = tf.constant(labels_pairs[:7000])
y_test = tf.constant(labels_pairs[7000:8500])
y_val = tf.constant(labels_pairs[8500:10000])
pair_train = (csi_pairs[:7000])
pair_test = (csi_pairs[7000:8500])
pair_val = (csi_pairs[8500:10000])

In [21]:
pair_train.shape, pair_test.shape, pair_val.shape

((7000, 2, 200, 30), (1500, 2, 200, 30), (1500, 2, 200, 30))

In [22]:
y_train.shape, y_test.shape, y_val.shape

(TensorShape([7000, 2, 1]),
 TensorShape([1500, 2, 1]),
 TensorShape([1500, 2, 1]))

In [23]:
y_train = np.squeeze(y_train)
y_test = np.squeeze(y_test)
y_val = np.squeeze(y_val)


In [24]:
y_train = y_train-1

In [25]:
y_test = y_test-1
y_val = y_val-1

# creating the model

In [26]:
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, MaxPooling1D, GlobalAveragePooling1D, GlobalMaxPooling1D, Dense, concatenate
from tensorflow.keras.models import Model

def encoder():
  inp = Input(shape = (200, 30))
  c1 = Conv1D(filters = 64, kernel_size = 5, activation = 'relu')(inp)    #   10,8n
  b1 = BatchNormalization()(c1)
  m1 = MaxPooling1D(2)(b1)
  c2 = Conv1D(filters = 128, kernel_size = 5, activation = 'relu')(m1)    #   8,8nDense(512, activation = 'relu')(inp)
  b2 = BatchNormalization()(c2)
  m2 = MaxPooling1D(2)(b2)

  f_avg = GlobalAveragePooling1D()(m2)
  f_max = GlobalMaxPooling1D()(m2)

  pool = tf.keras.layers.concatenate([f_avg, f_max])
  out = Dense(512, activation = 'relu')(pool)
  model = tf.keras.Model(inputs = inp, outputs = out)
  model.summary()
  return model

def classifier(emb_d):
  inp = Input(shape = (emb_d))
  d1 = Dense(300, activation = 'relu')(inp)
  out = Dense(276, activation = 'sigmoid')(d1)
  model = tf.keras.Model(inputs = inp, outputs = out)
  model.summary()

  return model


print("[INFO] building siamese network...")
dataS = Input(shape = (200, 30))   #   104
dataT = Input(shape = (200, 30))

encoder_common = encoder()

embedding_source = encoder_common([dataS])#([dataS, pos_encod_inp])
embedding_target = encoder_common([dataT])#([dataT, pos_encod_inp])

concat = tf.keras.layers.concatenate([embedding_source, embedding_target])

#dim = 128
d = 512
act_classifier = classifier(emb_d = d)#embedding2coordinate_cnn()

embedding_s = concat[ : , : d ]
embedding_t = concat[ : , d : ]

act_s = act_classifier(embedding_s)
act_t = act_classifier(embedding_t)

act_cat = tf.keras.layers.concatenate([act_s, act_t])
act_out_s = act_cat[ : , : 276 ]
act_out_t = act_cat[ : , 276 : ]

#model = Model(inputs = [dataA, dataB], outputs = [coord_out_a, coord_out_b, mmd])
model = Model(inputs = [dataS, dataT], outputs = [act_out_s, act_out_t])  #, S_identity, T_identity , recon_S, recon_T

model.summary()

[INFO] building siamese network...
Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_7 (InputLayer)        [(None, 200, 30)]            0         []                            
                                                                                                  
 conv1d_2 (Conv1D)           (None, 196, 64)              9664      ['input_7[0][0]']             
                                                                                                  
 batch_normalization_2 (Bat  (None, 196, 64)              256       ['conv1d_2[0][0]']            
 chNormalization)                                                                                 
                                                                                                  
 max_pooling1d_2 (MaxPoolin  (None, 98, 64)              

In [27]:
opt = tf.keras.optimizers.legacy.Adam(learning_rate = 0.001)
print("[INFO] compiling model...")
model.compile(loss = ["sparse_categorical_crossentropy", "sparse_categorical_crossentropy"], loss_weights = [1., 1.], optimizer= opt)
print('[INFO] building and compilation complete!')
print('\n Model summary:')
model.summary()

[INFO] compiling model...
[INFO] building and compilation complete!

 Model summary:
Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 200, 30)]            0         []                            
                                                                                                  
 input_6 (InputLayer)        [(None, 200, 30)]            0         []                            
                                                                                                  
 model_3 (Functional)        (None, 512)                  183104    ['input_5[0][0]',             
                                                                     'input_6[0][0]']             
                                                                                                  
 concat

In [28]:
computeUnits = .all

SyntaxError: invalid syntax (1047717362.py, line 1)

In [29]:
print("[INFO] training model...")

callback = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience=100, restore_best_weights=True)
history = model.fit(
    [pair_train[:, 0], pair_train[:, 1]], [y_train[:, 0] , y_train[:, 1] ],  #pairTrain[:, 1]
  validation_data = ([pair_val[:, 0], pair_val[:, 1]], y_val[:, 0], y_val[:, 1]),
    epochs=100
    ,batch_size = 100    #8192
  , callbacks=callback
    )

[INFO] training model...
Epoch 1/100
70/70 [==============================] - 5s 60ms/step - loss: 7.9948 - tf.__operators__.getitem_6_loss: 2.8729 - tf.__operators__.getitem_7_loss: 5.1220 - val_loss: 945.0135 - val_tf.__operators__.getitem_6_loss: 312.9904 - val_tf.__operators__.getitem_7_loss: 632.0231
Epoch 2/100
70/70 [==============================] - 4s 56ms/step - loss: 4.7809 - tf.__operators__.getitem_6_loss: 0.1475 - tf.__operators__.getitem_7_loss: 4.6334 - val_loss: 529.9079 - val_tf.__operators__.getitem_6_loss: 190.5665 - val_tf.__operators__.getitem_7_loss: 339.3414
Epoch 3/100
70/70 [==============================] - 4s 55ms/step - loss: 4.6851 - tf.__operators__.getitem_6_loss: 0.0605 - tf.__operators__.getitem_7_loss: 4.6246 - val_loss: 584.1915 - val_tf.__operators__.getitem_6_loss: 152.0317 - val_tf.__operators__.getitem_7_loss: 432.1598
Epoch 4/100
70/70 [==============================] - 4s 55ms/step - loss: 4.6792 - tf.__operators__.getitem_6_loss: 0.0604 - tf._

KeyboardInterrupt: 

In [20]:
model.evaluate([pair_test[:, 0], pair_test[:, 1]], [y_test[:, 0] , y_test[:, 1] ])

47/47 [==============================] - 0s 7ms/step - loss: 7.5918 - tf.__operators__.getitem_2_loss: 2.9867 - tf.__operators__.getitem_3_loss: 4.6052


[7.591821670532227, 2.9866514205932617, 4.605167865753174]